In [3]:
# this is a from scratch implementation of a simple RNN having 2 hidden layers
# it reads in any text file, tries it to learn character by character and then generates sentences
# the basic code is taken from http://karpathy.github.io/2015/05/21/rnn-effectiveness/
data = open('BThesis.txt', 'r').read()
chars = list(set(data))
totalCharacters = len(data)
vocabLen = len(chars)

In [4]:
characterToIndex = {ch:i for i,ch in enumerate(chars)}
print(characterToIndex)
indexToCharacter = {i:ch for i,ch in enumerate(chars)}
print(indexToCharacter)

{'>': 0, 'f': 1, 'S': 2, 'd': 3, '”': 4, 'r': 5, 'R': 6, 'x': 7, '+': 8, '-': 9, '–': 10, 'l': 11, 'T': 12, '[': 13, 'F': 14, 's': 15, 'N': 16, 'O': 17, 'W': 18, 'z': 19, 'q': 20, 'I': 21, 'o': 22, '—': 23, 'P': 24, 'n': 25, 'k': 26, 'K': 27, '<': 28, 'a': 29, ' ': 30, ']': 31, 'h': 32, 'y': 33, 'i': 34, 'C': 35, 'w': 36, 'X': 37, 'u': 38, ':': 39, ',': 40, '/': 41, 'D': 42, 'm': 43, '’': 44, 'M': 45, '\n': 46, 'Y': 47, 'g': 48, 'Q': 49, '=': 50, 'e': 51, 'j': 52, 'c': 53, '_': 54, '“': 55, 'v': 56, 'A': 57, '.': 58, 'U': 59, 'p': 60, 'b': 61, 't': 62, 'G': 63, 'V': 64, 'B': 65, 'L': 66, 'E': 67, 'H': 68}
{0: '>', 1: 'f', 2: 'S', 3: 'd', 4: '”', 5: 'r', 6: 'R', 7: 'x', 8: '+', 9: '-', 10: '–', 11: 'l', 12: 'T', 13: '[', 14: 'F', 15: 's', 16: 'N', 17: 'O', 18: 'W', 19: 'z', 20: 'q', 21: 'I', 22: 'o', 23: '—', 24: 'P', 25: 'n', 26: 'k', 27: 'K', 28: '<', 29: 'a', 30: ' ', 31: ']', 32: 'h', 33: 'y', 34: 'i', 35: 'C', 36: 'w', 37: 'X', 38: 'u', 39: ':', 40: ',', 41: '/', 42: 'D', 43: 'm', 

In [5]:
import numpy as np
#hyperparameters
learningRate = 0.01
hiddenLayer = 256
seqLength = 50
#modelParameters
#connect input layer to first hidden layer
W1 = np.random.randn(vocabLen, hiddenLayer) * 0.01
# connect second hidden layer to output layer
W2 = np.random.randn(hiddenLayer, vocabLen) * 0.01
# connect first hidden layer to first hidden layer in the next timestamp
Wh1 = np.random.randn(hiddenLayer, hiddenLayer) * 0.01
# connect the first hidden layer to the second hidden layer
Whh = np.random.randn(hiddenLayer, hiddenLayer) * 0.01
# connect the second hidden layer to the second hidden layer in the next timestamp
Wh2 = np.random.randn(hiddenLayer, hiddenLayer) * 0.01
# bias for W1
b1 = np.random.randn(hiddenLayer, 1)
# bias for Whh
bh = np.random.randn(hiddenLayer, 1)
# bias for W2
b2 = np.random.randn(vocabLen, 1)

In [6]:
# the loss function would take in the input chars, the output chars and the previous hidden state
# it outputs the hidden state, the gradients for each parameter between layers and the last hidden states
def propagate(inputChars, outputChars, prevH, prevH1):
    x, h, h1, y, p = {}, {}, {}, {}, {}
    #x = the array which is a list of zeros, with just 1 at the index where input character is
    #h = values of hidden layers at different times
    #y = values of outputs not activated
    #p = activated output
    h[-1] = np.copy(prevH)
    h1[-1] = np.copy(prevH1)
    loss = 0
    #forward propagation
    for t in range(len(inputChars)):
        x[t] = np.zeros((vocabLen, 1))
        x[t][inputChars[t]] = 1
        h[t] = np.tanh(W1.T.dot(x[t]) + Wh1.dot(h[t-1]) + b1) # h has to be of the same dimension as b
        h1[t] = np.tanh(Whh.T.dot(h[t]) + Wh2.dot(h1[t-1]) + bh)
        y[t] = W2.T.dot(h1[t]) + b2
        p[t] = np.exp(y[t])/np.sum(np.exp(y[t]))
        loss += -np.log(p[t][outputChars[t],0])
    
    
    #backward propagation
    dW1, dW2, dWh1, dWhh, dWh2 = np.zeros_like(W1), np.zeros_like(W2), np.zeros_like(Wh1), np.zeros_like(Whh), np.zeros_like(Wh2)
    db1, db2, dbh = np.zeros_like(b1), np.zeros_like(b2), np.zeros_like(bh)
    dhnext = np.zeros_like(h[0])
    dh1next = np.zeros_like(h1[0])
    for t in reversed(range(len(inputChars))):
        dy = np.copy(p[t])
        #starting the backpropagation
        dy[outputChars[t]] -= 1
        
        dW2 += h1[t].dot(dy.T)
        db2 += dy
        #generally the error is backpropageted by multiplying the error in the output to the input's transpose
        #here the transport is of the error, but this is because the way i have initialized the shape of the matrices
        #in the original implementation, it is the standard way
        
        #second hidden layer backpropagation
        dh1 = np.dot(W2, dy) + dh1next
        #we got the below line of code by differentiation of the activation function (tanh)
        dh1raw = (1 - h1[t] * h1[t]) * dh1
        # so basically the activation is differentiated in two steps
        # first is the values inside the tanh function (dh1), then finally the tanh function itself,
        # and as from the chain rule dh1 is multiplied to it
        dbh += dh1raw
        dWhh += np.dot(h[t], dh1raw.T)
        dWh2 += np.dot(dh1raw, h1[t-1].T)
        
        #first hidden layer backpropagation
        dh = np.dot(Whh, dh1) + dh1next
        dhraw = (1 - h[t] * h[t]) * dh
        db1 += dhraw
        dW1 += np.dot(x[t], dhraw.T) #derivative of input to hidden layer weight
        dWh1 += np.dot(dhraw, h[t-1].T) #derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Wh1.T, dhraw)
    for dparam in [dW1, dWh1, dWhh, dWh2, dW2, db1, dbh, db2]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dW1, dWh1, dWhh, dWh2, dW2, db1, dbh, db2, h[len(inputChars)-1]
    

In [23]:
#prediction, one full forward pass
def sample(h, h1, seed, n):
                                                                                                                                                                                        
    #sample a sequence of integers from the model                                                                                                                                                
    #h is memory state, seed is seed letter for first time step   
    #n is how many characters to predict

    x = np.zeros((vocabLen, 1))
    x[seed] = 1
    #list to store generated chars
    outputChars = []
    for t in range(n):
        h = np.tanh(np.dot(W1.T, x) + np.dot(Wh1, h) + b1)
        h1 = np.tanh(np.dot(Whh.T, h) + np.dot(Wh2, h1) + bh)
        #compute output (unnormalised)
        y = np.dot(W2.T, h1) + b2
        # probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #print(p)
        #pick one with the highest probability 
        selectedChar = np.random.choice(range(vocabLen), p=p.ravel())
        #print(ix)
        #create a vector
        x = np.zeros((vocabLen, 1))
        #customize it for the predicted char
        x[selectedChar] = 1
        #add it to the list
        outputChars.append(selectedChar)

    txt = ''.join(indexToCharacter[char] for char in outputChars)
    print ('----\n %s \n----' % (txt, ))
    hprev = np.zeros((hiddenLayer,1)) # reset RNN memory  
    h1prev = np.zeros((hiddenLayer,1)) # reset RNN memory 
    #predict the 200 next characters given 'a'
sample(hprev,h1prev,characterToIndex['a'],200)

----
 te recommender system: as trid s thete, incoud skere information lion of mimit d, to to matrix when we added to crevic be of tambed Sendiecality have that I had thic lould be user nationsed us to sear 
----


In [19]:
n, p = 0, 0
mW1, mWh1, mWhh, mWh2, mW2 = np.zeros_like(W1), np.zeros_like(Wh1), np.zeros_like(Whh), np.zeros_like(Wh2), np.zeros_like(W2)
mb1, mbh, mb2 = np.zeros_like(b1), np.zeros_like(bh), np.zeros_like(b2) # memory variables for Adagrad                                                                                                                
smoothLoss = -np.log(1.0/vocabLen)*seqLength # loss at iteration 0                                                                                                                        
while n<=1000*100:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # check "How to feed the loss function to see how this part works
    if p+seqLength+1 >= len(data) or n == 0:
        hprev = np.zeros((hiddenLayer,1)) # reset RNN memory   
        h1prev = np.zeros((hiddenLayer,1))
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [characterToIndex[ch] for ch in data[p:p+seqLength]]
    targets = [characterToIndex[ch] for ch in data[p+1:p+seqLength+1]]

    # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dW1, dWh1, dWhh, dWh2, dW2, db1, dbh, db2, hprev = propagate(inputs, targets, hprev, h1prev)
    smoothLoss = smoothLoss * 0.999 + loss * 0.001

    # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print ('iter %d, loss: %f' % (n, smoothLoss)) # print progress
        sample(hprev, h1prev, inputs[0], 200)

    # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([W1, Wh1, Whh, Wh2, W2, b1, bh, b2],
    [dW1, dWh1, dWhh, dWh2, dW2, db1, dbh, db2],
    [mW1, mWh1, mWhh, mWh2, mW2, mb1, mbh, mb2]):
        mem += dparam * dparam
        param += -learningRate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seqLength # move data pointer                                                                                                                                                         
    n += 1 # iteration counter

iter 0, loss: 211.514828
----
  usuge difees as to n is that to suedses way morefstived as additional to learn, the apto pred.

Adir, Filtering. The mading ablitid sparse data. Buth acherefore we need. In to a form ardedict in of t 
----
iter 1000, loss: 128.568182
----
 nfror of sto sest Thith bervioul information and co. Loviouct in towe . ]G fove sigris an ompes tive sems us tu omore seormse woice of a so move gilter trix of t ives coul best in the in a movien this 
----
iter 2000, loss: 83.097862
----
 a fried if a gurt . when we addes or folthes wate  osed eatece to eecels and clesention, ar ititially. The fir allystesprally go calculatent imally enterselt. We woweldasus alrowithrs. The ghow whe th 
----
iter 3000, loss: 62.130336
----
 tion hechncensty. There gromptilit is to theithm to add to M tenser mode approady the user, Pequstict of thowherged . This ourgow ition torm approach boove watrix factorization methochibes  ore berm i 
----
iter 4000, loss: 52.206116
----
  goo tee